In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
import math




pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#read data from file
ratings = pd.read_csv("./ml-100k/u.data",sep="\t",names = ["user_id", "item_id" , "ratings", "timestamp"])


genres = pd.read_csv("./ml-100k/u.item",sep="|",names = ["movie_id", "movie_title",\
            "release date", "video release date", "IMDb URL", "unknown", "Action",\
            "Adventure","Animation", "Children's", "Comedy", "Crime", "Documentary",\
            "Drama","Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",\
            "Romance","Sci-Fi", "Thriller", "War", "Western"], encoding='latin-1')



In [37]:
tab_user = ratings.pivot_table(columns = ["item_id"] , index = ["user_id"], values = "ratings")#fill_value=0)
#centered cosine
#It finds mean of each row and substract elementwise

user_A = tab_user
user_A = user_A.fillna(0)
user_A = np.array(user_A)

In [ ]:
def NonNegativemat(user_A,f=100,lamW=0.001,lamH=0.001, steps = 100):
    #m is the no of users
    #n is no of items
    # fine?
    #Step1
    m,n = user_A.shape
    ruihat = np.zeros((m,n))
    W=np.random.rand(m,f)
    H=np.random.rand(f,n)
    gateActivated = False

    #Step2
    wup=np.zeros([m,f])
    wdn=np.zeros([m,f])
    hup=np.zeros([f,n])
    hdn=np.zeros([f,n])
    #Step 3 : repeat
    #while(e > 0.001):
    step_list=[]
    for step in range(steps):
        #Step 4
        wup=np.zeros((m,f))
        wdn=np.zeros((m,f))
        hup=np.zeros((f,n))
        hdn=np.zeros((f,n))
        #Step 5
        if step > 0:
            print("step = ", step, " error = ", norm_ei)
            step_list.append(norm_ei)

        for u in range(0,m):
            for i in range(0,n):
                if user_A[u][i] > 0:
                    #Step6
                    ruihat[u][i] = np.dot(W[u,:],H[:, i])
                    #Step7
                    sum_ei = 0
                    #Step8
                    for v in range(m):
                        if user_A[v][i] > 0:
                            #Step 9
                            sum_ei += (user_A[v][i] - np.dot(W[v,:],H[:, i])) ** 2
                    #Step 10 ends for Step 8

                    #Step11
                    norm_ei = math.sqrt(sum_ei)
                    #Step 12:
                    for k in range (1, f):
                        
                        #Step 13,14,15,16
                        try:
                            wup[u][k] = float(wup[u][k] + H[k][i] * user_A[u][i]) / norm_ei
                            wdn[u][k] = float(wdn[u][k] + H[k][i] * ruihat[u][i])/ norm_ei
                            hup[k][i] = float(hup[k][i] + W[u][k] * user_A[u][i]) / norm_ei
                            hdn[k][i] = float(hdn[k][i] + W[u][k] * ruihat[u][i]) / norm_ei
                        except ZeroDivisionError:
                            return ruihat,prev_norm,step_list

                      
                    #if norm_ei ==0:
                        #print(norm_ei)
                    
                    #Step 17 end for Step 12

        #Step 18 end for step 5
        #Step 19
        for u in range(m):
            total_movies_rated =sum(ii > 0 for ii in user_A[u])
            if (total_movies_rated > 0):
                #Step 20
                for k in range (1, f):
                    #Step 21
                    try:
                        W[u][k] = float(W[u][k] * wup[u][k])/float(wdn[u][k] + total_movies_rated * lamW * W[u][k])
                    except ZeroDivisionError:
                        return ruihat,prev_norm,step_list

                #Step 22 End for Step 20
        #print(np.min(wdn),np.min(W))
        # Step 23 end for Step 19
 
        #Step 24:
        
        
        for i in range(n):
            total_users_rated =sum(ii > 0 for ii in user_A[:, i])
            if (total_users_rated > 0):
                #Step 25
                for k in range (1, f):
                    #Step 26
                    try:
                        H[k][i] =float (H[k][i] * hup[k][i])/float(hdn[k][i] + total_users_rated * lamH * H[k][i])
                    except ZeroDivisionError:
                        return ruihat,prev_norm,step_list
        for i in range(m):
            for j in range(n):
                ruihat[i][j] = np.dot(W[i,:],H[:,j])
        prev_norm=norm_ei
        
            #Step 27 End for Step 25
        #print(np.min(hdn),np.min(H))
    # Step 28 end for Step 24
    #STep29 end for Step 3

    #Step 30
    
    return ruihat,norm_ei,step_list

In [ ]:
mat,error,step = NonNegativemat(user_A)

step =  1  error =  48.73069430912503
step =  2  error =  4.543302916020331
step =  3  error =  5.240939606398964
step =  4  error =  2.506699853079297
step =  5  error =  2.4159843489154733
step =  6  error =  1.8217190092745474
step =  7  error =  1.4457603846534126
step =  8  error =  1.1927251365999987
step =  9  error =  0.8902143282933259
step =  10  error =  0.7841213281057922
step =  11  error =  0.6834749814898289
step =  12  error =  0.5700705274582974
step =  13  error =  0.6585981139177616
step =  14  error =  0.4772099645530462
step =  15  error =  0.5566976381536076
step =  16  error =  0.4152881482436237
step =  17  error =  0.3824926782656448
step =  18  error =  0.34111029508266155
step =  19  error =  0.2675922737474564
step =  20  error =  0.3372268399704939
step =  21  error =  0.22889458317631312
step =  22  error =  0.3422323218770273
step =  23  error =  0.21520660630114558
step =  24  error =  0.32579949001130537
step =  25  error =  0.20392018728625963
step =  

In [29]:

user_id = 117
recommendations = []
lowest = -float('Inf')

for i in range(len(mat[user_id])):
    if len(recommendations) < 10:
        recommendations.append((mat[user_id][i],i))
        recommendations.sort(key=lambda x: x[0],reverse = True)
        lowest = recommendations[-1][0]
    elif(mat[user_id][i] > lowest):
        recommendations.append((mat[user_id][i],i))
        recommendations.sort(key=lambda x: x[0],reverse = True)
        recommendations = recommendations[:10]
        lowest = recommendations[-1][0]

print('Top 10 Recommended Movies for userId:',user_id)
for i in range(len(recommendations)):
    print(i + 1,genres.loc[recommendations[i][1]].movie_title)

[[0.20473252 0.20347943 0.40981535 ... 0.25596594 0.46760158 0.45543101]
 [0.62370098 1.84233567 2.55014573 ... 2.42125615 3.34720875 3.62277161]
 [0.83968245 1.51618587 2.37572871 ... 1.96883251 2.79171987 3.00534211]
 ...
 [0.89658772 2.19035192 3.1868199  ... 3.29509399 4.07979064 4.36354646]
 [0.58806939 1.70965498 2.3773136  ... 2.24491548 3.11902213 3.37312909]
 [0.79163997 2.35816358 2.51802398 ... 2.21022419 3.05169787 3.42121248]]
Top 10 Recommended Movies for userId: 117
1 Safe Passage (1994)
2 They Made Me a Criminal (1939)
3 Saint of Fort Washington, The (1993)
4 Someone Else's America (1995)
5 Crossfire (1947)
6 Substance of Fire, The (1996)
7 Tough and Deadly (1995)
8 Jupiter's Wife (1994)
9 Mina Tannenbaum (1994)
10 Prefontaine (1997)
